In [17]:
from imutils import paths
import numpy as np
import imutils
import cv2

In [18]:
def calculate_focal_length(known_distance, known_width, width_in_rf_image):
    return (width_in_rf_image * known_distance) / known_width

def calculate_distance(focal_length, known_width, width_in_frame):
    return (known_width * focal_length) / width_in_frame

def capture_card_template():
    print("Align the card within the box and press 'c' to capture the card template.")
    while True:
        ret, frame = cap.read()
        
        # Draw a guide frame on the screen
        height, width, _ = frame.shape
        cv2.rectangle(frame, (int(width*0.3), int(height*0.3)), (int(width*0.7), int(height*0.7)), (0, 255, 0), 2)
        cv2.putText(frame, "Press 'c' to capture card template", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Capture Card Template', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            # Capture the template
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Crop the region inside the guide frame
            card_template = gray_frame[int(height*0.3):int(height*0.7), int(width*0.3):int(width*0.7)]
            cv2.destroyWindow('Capture Card Template')
            return card_template

def detect_card_with_template(frame, template):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Ensure the template is smaller than the frame
    if template.shape[0] > gray_frame.shape[0] or template.shape[1] > gray_frame.shape[1]:
        raise ValueError("Template image is larger than the frame. Resize the template.")

    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    if max_val > 0.7:  # Threshold to consider it a match
        (tH, tW) = template.shape[:2]
        top_left = max_loc
        bottom_right = (top_left[0] + tW, top_left[1] + tH)
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        return tW  # Return the width of the template in pixels
    return None


In [19]:
import cv2
import numpy as np

# Constants
KNOWN_WIDTH_OBJ = 8.5  # Known width of the reference object (e.g., credit card in cm)
KNOWN_DISTANCE = 25.5  # Calibration distance (in cm)
ALERT_DISTANCE = 50.0  # Distance threshold for alert (in cm)

# Start video capture
cap = cv2.VideoCapture(0)

# Capture the card template during the calibration step
card_template = capture_card_template()

# Calibration step
print("Please hold the card in front of the camera as before, and align it within the frame box.")

calibrated = False
focal_length = None

while not calibrated:
    ret, frame = cap.read()
    
    # Draw a guide frame on the screen
    height, width, _ = frame.shape
    cv2.rectangle(frame, (int(width*0.3), int(height*0.3)), (int(width*0.7), int(height*0.7)), (0, 255, 0), 2)
    cv2.putText(frame, "Press 'c' to calibrate", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    cv2.imshow('Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):  # Press 'c' to calibrate
        object_width_in_frame = detect_card_with_template(frame, card_template)
        
        if object_width_in_frame:
            # Calculate the focal length using the width of the detected card
            focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH_OBJ, object_width_in_frame)
            print(f"Calibration complete. Focal Length calculated: {focal_length:.2f} pixels.")
            calibrated = True
        else:
            print("Object not detected. Please ensure the card is visible and try again.")
    
    if key == ord('q'):  # Press 'q' to quit calibration
        print("Calibration aborted.")
        cap.release()
        cv2.destroyAllWindows()
        exit()

cv2.destroyWindow('Calibration')  # Close the calibration window

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        #print("Face Width",w)  ##################################################
        distance = calculate_distance(focal_length, KNOWN_WIDTH_OBJ, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Check if the distance exceeds the alert threshold
        if distance > ALERT_DISTANCE:
            cv2.putText(frame, "Alert: User left the desk!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow('Distance Monitor', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


Align the card within the box and press 'c' to capture the card template.
Please hold the card in front of the camera as before, and align it within the frame box.
Calibration complete. Focal Length calculated: 768.00 pixels.


In [12]:
import cv2
import numpy as np

focal_length = 768.0
KNOWN_WIDTH_OBJ = 8.5  # Known width of the reference object (e.g., credit card in cm)
KNOWN_DISTANCE = 25.5  # Calibration distance (in cm)
ALERT_DISTANCE = 50.0  # Distance threshold for alert (in cm)

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        #print("Face Width",w)  ##################################################
        distance = calculate_distance(focal_length, KNOWN_WIDTH_OBJ, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Check if the distance exceeds the alert threshold
        if distance > ALERT_DISTANCE:
            cv2.putText(frame, "Alert: User left the desk!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow('Distance Monitor', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [8]:
import cv2
import face_recognition

# Load a sample picture and learn how to recognize it.
image = face_recognition.load_image_file("Downloads/EMS_desk/desk1/WIN_20240729_18_29_33_Pro.jpg")
face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)

print(f"Detected {len(face_encodings)} face(s)")

# Display the result for debugging
if len(face_encodings) > 0:
    print("Face encoding successful.")
else:
    print("No faces detected or encoding failed.")


Detected 1 face(s)
Face encoding successful.


In [ ]:
                                ## HUMAN TRIALS ## BEWARE ## DANGER ## BEWARE ## HUMAN TRIALS ##

In [20]:
import cv2
import numpy as np
import time

def calculate_focal_length(known_distance, known_width, width_in_rf_image):
    return (width_in_rf_image * known_distance) / known_width

def calculate_distance(focal_length, known_width, width_in_frame):
    return (known_width * focal_length) / width_in_frame

def capture_face_template():
    print("Align your face within the oval and press 'c' to capture the face template.")
    while True:
        ret, frame = cap.read()
        
        height, width, _ = frame.shape
        center = (int(width * 0.5), int(height * 0.5))
        axes = (int(width * 0.2), int(height * 0.3))  # Ellipse axes lengths
        
        # Draw an oval guide on the screen
        cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'c' to capture face template", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Capture Face Template', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face_template = gray_frame[center[1] - axes[1]:center[1] + axes[1], center[0] - axes[0]:center[0] + axes[0]]
            cv2.destroyWindow('Capture Face Template')
            return face_template

def detect_face_with_template(frame, template):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if template.shape[0] > gray_frame.shape[0] or template.shape[1] > gray_frame.shape[1]:
        raise ValueError("Template image is larger than the frame. Resize the template.")

    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    if max_val > 0.7:
        (tH, tW) = template.shape[:2]
        top_left = max_loc
        bottom_right = (top_left[0] + tW, top_left[1] + tH)
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        return tW
    return None


KNOWN_WIDTH_OBJ = 15.0  # Known width of the human face in cm
KNOWN_DISTANCE = 40.0
ALERT_DISTANCE = 50.0

In [21]:
import cv2
import numpy as np

# Replace the card calibration with face calibration
cap = cv2.VideoCapture(0)
face_template = capture_face_template()

print("Please align your face within the oval and press 'c' to calibrate.")

calibrated = False
focal_length = None

while not calibrated:
    ret, frame = cap.read()
    
    height, width, _ = frame.shape
    center = (int(width * 0.5), int(height * 0.5))
    axes = (int(width * 0.2), int(height * 0.3))
    cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)
    cv2.putText(frame, "Press 'c' to calibrate", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    cv2.imshow('Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        face_width_in_frame = detect_face_with_template(frame, face_template)
        
        if face_width_in_frame:
            focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH_OBJ, face_width_in_frame)
            print(f"Calibration complete. Focal Length calculated: {focal_length:.2f} pixels.")
            calibrated = True
        else:
            print("Face not detected. Please ensure your face is visible and try again.")
    
    if key == ord('q'):
        print("Calibration aborted.")
        cap.release()
        cv2.destroyAllWindows()
        exit()

cv2.destroyWindow('Calibration')
cap.release()

Align your face within the oval and press 'c' to capture the face template.
Please align your face within the oval and press 'c' to calibrate.
Calibration complete. Focal Length calculated: 682.67 pixels.


In [25]:
alert_start_time = None
focal_length = 682.67
ALERT_DISTANCE = 70.0

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        distance = calculate_distance(focal_length, KNOWN_WIDTH_OBJ, w)

        cv2.putText(frame, f"Distance: {distance:.2f} cm", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        if distance > ALERT_DISTANCE:
            if alert_start_time is None:
                alert_start_time = time.time()  # Start timing when distance exceeds the threshold
            elif time.time() - alert_start_time > 5:  # Check if 1 minute has passed
                cv2.putText(frame, "Alert: User left the desk!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            alert_start_time = None  # Reset timer if the user moves back within the distance

    cv2.imshow('Distance Monitor', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [22]:
cap.release()

In [27]:
import cv2
import numpy as np
import time

def calculate_focal_length(known_distance, known_width, width_in_rf_image):
    return (width_in_rf_image * known_distance) / known_width

def calculate_distance(focal_length, known_width, width_in_frame):
    return (known_width * focal_length) / width_in_frame

def capture_face_template(cap):
    print("Align your face within the oval and press 'c' to capture the face template.")
    while True:
        ret, frame = cap.read()
        
        height, width, _ = frame.shape
        center = (int(width * 0.5), int(height * 0.5))
        axes = (int(width * 0.2), int(height * 0.3))  # Ellipse axes lengths
        
        cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'c' to capture face template", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Capture Face Template', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face_template = gray_frame[center[1] - axes[1]:center[1] + axes[1], center[0] - axes[0]:center[0] + axes[0]]
            cv2.destroyWindow('Capture Face Template')
            return face_template

def detect_face_with_template(frame, template):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if template.shape[0] > gray_frame.shape[0] or template.shape[1] > gray_frame.shape[1]:
        raise ValueError("Template image is larger than the frame. Resize the template.")

    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)

    if max_val > 0.7:
        (tH, tW) = template.shape[:2]
        top_left = max_loc
        bottom_right = (top_left[0] + tW, top_left[1] + tH)
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        return tW
    return None

def calibrate_focal_length(cap, known_distance, known_width_obj, face_template):
    print("Please align your face within the oval and press 'c' to calibrate.")
    while True:
        ret, frame = cap.read()
        
        height, width, _ = frame.shape
        center = (int(width * 0.5), int(height * 0.5))
        axes = (int(width * 0.2), int(height * 0.3))
        cv2.ellipse(frame, center, axes, 0, 0, 360, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'c' to calibrate", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Calibration', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            face_width_in_frame = detect_face_with_template(frame, face_template)
            
            if face_width_in_frame:
                focal_length = calculate_focal_length(known_distance, known_width_obj, face_width_in_frame)
                print(f"Calibration complete. Focal Length calculated: {focal_length:.2f} pixels.")
                cv2.destroyWindow('Calibration')
                return focal_length
            else:
                print("Face not detected. Please ensure your face is visible and try again.")
        
        if key == ord('q'):
            print("Calibration aborted.")
            cap.release()
            cv2.destroyAllWindows()
            exit()

def monitor_distance(cap, focal_length, known_width_obj, alert_distance):
    alert_start_time = None
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            distance = calculate_distance(focal_length, known_width_obj, w)

            cv2.putText(frame, f"Distance: {distance:.2f} cm", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            if distance > alert_distance:
                if alert_start_time is None:
                    alert_start_time = time.time()  # Start timing when distance exceeds the threshold
                elif time.time() - alert_start_time > 5:  # Check if 5 seconds have passed
                    cv2.putText(frame, "Alert: User left the desk!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                alert_start_time = None  # Reset timer if the user moves back within the distance

        cv2.imshow('Distance Monitor', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def main():
    KNOWN_WIDTH_OBJ = 15.0  # Known width of the human face in cm
    KNOWN_DISTANCE = 40.0
    ALERT_DISTANCE = 70.0

    cap = cv2.VideoCapture(0)
    
    face_template = capture_face_template(cap)
    
    focal_length = calibrate_focal_length(cap, KNOWN_DISTANCE, KNOWN_WIDTH_OBJ, face_template)
    
    # Set up a new capture for monitoring
    cap.release()
    cap = cv2.VideoCapture(0)
    
    monitor_distance(cap, focal_length, KNOWN_WIDTH_OBJ, ALERT_DISTANCE)
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Align your face within the oval and press 'c' to capture the face template.
Please align your face within the oval and press 'c' to calibrate.
Calibration complete. Focal Length calculated: 682.67 pixels.
